### Weather Service App

 - We will run the simple weather service application on K8s using Minikube. It will involve building a Docker image of the flask app, similar to before.

 - The single node cluster can be started with the following command:

 ```bash
 minikube start
 ```
 You will see the following blurb which ends with 'Done! kubectl is now configured to use minikube by default' the first time you run it. You can stop minikube using `minikube stop` after the whole exercise is done.

 ```bash
(datasci-dev) ttmac:k8s theja$ minikube start
😄  minikube v1.13.0 on Darwin 10.14.6
✨  Automatically selected the hyperkit driver
💾  Downloading driver docker-machine-driver-hyperkit:
    > docker-machine-driver-hyperkit.sha256: 65 B / 65 B [---] 100.00% ? p/s 0s
    > docker-machine-driver-hyperkit: 10.90 MiB / 10.90 MiB  100.00% 7.97 MiB p
🔑  The 'hyperkit' driver requires elevated permissions. The following commands will be executed:

    $ sudo chown root:wheel /Users/theja/.minikube/bin/docker-machine-driver-hyperkit
    $ sudo chmod u+s /Users/theja/.minikube/bin/docker-machine-driver-hyperkit


Password:
💿  Downloading VM boot image ...
    > minikube-v1.13.0.iso.sha256: 65 B / 65 B [-------------] 100.00% ? p/s 0s
    > minikube-v1.13.0.iso: 173.73 MiB / 173.73 MiB [ 100.00% 11.61 MiB p/s 15s
👍  Starting control plane node minikube in cluster minikube
💾  Downloading Kubernetes v1.19.0 preload ...
    > preloaded-images-k8s-v6-v1.19.0-docker-overlay2-amd64.tar.lz4: 486.28 MiB
🔥  Creating hyperkit VM (CPUs=2, Memory=4000MB, Disk=20000MB) ...
🐳  Preparing Kubernetes v1.19.0 on Docker 19.03.12 ...
🔎  Verifying Kubernetes components...
🌟  Enabled addons: default-storageclass, storage-provisioner
🏄  Done! kubectl is now configured to use "minikube" by default
 ```

 - If you run the status command, you should see the following:

```bash
(datasci-dev) ttmac:k8s theja$ minikube status
minikube
type: Control Plane
host: Running
kubelet: Running
apiserver: Running
kubeconfig: Configured
```

 - We can also query the nodes (we only have one) and more information about them as below:

```bash
(datasci-dev) ttmac:k8s theja$ kubectl get nodes
NAME       STATUS   ROLES    AGE    VERSION
minikube   Ready    master   131m   v1.19.0
(datasci-dev) ttmac:k8s theja$ kubectl get all
NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   140m
```

 - Although we have the weather_service image (can be viewed by using the `docker images` command as long as the Docker daemon is running) in our local repository, we need to access and use the repository specific to Minikube. This can be done using the following two commands:


```bash
(datasci-dev) ttmac:docker-weather-service theja$ minikube docker-env
(datasci-dev) ttmac:docker-weather-service theja$ eval $(minikube -p minikube docker-env)
```
 - Now if we build the same weather service flask app (recall the corresponding Dockerfile and weather.py from before). This time, we will give a slightly different repository name.

```bash
(datasci-dev) ttmac:docker-weather-service theja$ docker build -t weather-service-k8s/latest .
```

 - The newly created repository is now available in the minikube specific image repository (notice the other images for instance):

```bash
(datasci-dev) ttmac:docker-weather-service theja$ docker images
REPOSITORY                                TAG                 IMAGE ID            CREATED             SIZE
weather-service-k8s/latest                latest              ae4f44a22535        11 seconds ago      496MB
debian                                    buster-slim         052664ad4351        6 days ago          69.2MB
gcr.io/k8s-minikube/storage-provisioner   v3                  bad58561c4be        2 weeks ago         29.7MB
k8s.gcr.io/kube-proxy                     v1.19.0             bc9c328f379c        2 weeks ago         118MB
k8s.gcr.io/kube-apiserver                 v1.19.0             1b74e93ece2f        2 weeks ago         119MB
k8s.gcr.io/kube-controller-manager        v1.19.0             09d665d529d0        2 weeks ago         111MB
k8s.gcr.io/kube-scheduler                 v1.19.0             cbdc8369d8b1        2 weeks ago         45.7MB
k8s.gcr.io/etcd                           3.4.9-1             d4ca8726196c        2 months ago        253MB
kubernetesui/dashboard                    v2.0.3              503bc4b7440b        2 months ago        225MB
k8s.gcr.io/coredns                        1.7.0               bfe3a36ebd25        3 months ago        45.2MB
kubernetesui/metrics-scraper              v1.0.4              86262685d9ab        5 months ago        36.9MB
k8s.gcr.io/pause                          3.2                 80d28bedfe5d        7 months ago        683kB
```

 - Although there is a local image, we need to set a specific parameter to make kubectl not search for the image elsewhere. In order to do so, we create the following yaml (a data-serialization language thats quite popular to set config parameters and is very tab/space sensitive). See [https://en.wikipedia.org/wiki/YAML](https://en.wikipedia.org/wiki/YAML) for more info on yaml files.

 - The yaml was generated automatically by running the following command and then modified by adding the line `imagePullPolicy: Never`:

```bash
(datasci-dev) ttmac:docker-weather-service theja$ kubectl create deployment weather-minikube --image=weather-service-k8s:latest -o yaml --dry-run=client
```

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  creationTimestamp: null
  labels:
    app: weather-minikube
  name: weather-minikube
spec:
  replicas: 1
  selector:
    matchLabels:
      app: weather-minikube
  strategy: {}
  template:
    metadata:
      creationTimestamp: null
      labels:
        app: weather-minikube
    spec:
      containers:
      - image: weather-service-k8s/latest:latest
        name: weather-service-k8s
        resources: {}
        imagePullPolicy: Never
status: {}
```

 - Save the above in a file called. `weather_minikube.yaml`. Next we create a new deployment using the minikube cluster.

```bash
(datasci-dev) ttmac:docker-weather-service theja$ kubectl apply -f weather_minikube.yaml
deployment.apps/weather-minikube created
(datasci-dev) ttmac:docker-weather-service theja$ kubectl get all
NAME                                   READY   STATUS         RESTARTS   AGE
pod/weather-minikube-56fd45dd59-zkmmg   1/1     Running   0          71s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   160m

NAME                               READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/weather-minikube   1/1     1            1           71s

NAME                                         DESIRED   CURRENT   READY   AGE
replicaset.apps/weather-minikube-76cb958c4   1         1         1       71s
```

 - Next, we will expose this deployment. The option `--type=NodePort` specifies the type of the Service.

```bash
(datasci-dev) ttmac:docker-weather-service theja$ kubectl expose deployment weather-minikube --type=NodePort --port=5000
service/weather-minikube exposed
```

 - We can check that the container is in fact running fine using the following command:

```bash
(datasci-dev) ttmac:docker-weather-service theja$ kubectl get pod
NAME                                READY   STATUS    RESTARTS   AGE
weather-minikube-56fd45dd59-zkmmg   1/1     Running   0          171m
```

 - Once we have verified that the container is running, we can figure out the URL that is being exposed by the system as follows:

```bash
(datasci-dev) ttmac:docker-weather-service theja$ minikube service weather-minikube --url
http://192.168.64.2:32233
```

 - We can then access the weather service flask app from the browser and make sure that it works with a few test examples:

![container_browser_local](images/container_browser_local.png)



 - We may have to debug our container for various reasons. For instance, it can throw up 500 errors, which correspond to internal server errors due to errors in the python function attached to a URL route in Flask. There are a couple of options:
   -  We can look at the logs (the last argument is the pod name and it has a single container in our example):

```bash
kubectl logs weather-minikube-56fd45dd59-zkmmg
```
   - We can access the container shell (assuming it has `bash` at the appropriate path) by referencing the pod name and try to see which commands are not working:

```bash
kubectl exec --stdin --tty weather-minikube-56fd45dd59-zkmmg -- /bin/bash
```

   - Finally, the command `kubectl describe` also helps in debugging by showing us detailed information associated with any resource.


 - Teardown involves the following commands:

 ```bash
(datasci-dev) ttmac:docker-weather-service theja$ kubectl delete services weather-minikube
service "weather-minikube" deleted
(datasci-dev) ttmac:docker-weather-service theja$ kubectl delete deployment weather-minikube
deployment.apps "weather-minikube" deleted
 ```

 - We can then stop and delete the minikube cluster using the ` minikube stop` and `minikube delete` commands:

```bash
(datasci-dev) ttmac:docker-weather-service theja$ minikube stop
✋  Stopping node "minikube"  ...
🛑  1 nodes stopped.
``` 

 - Finally, you can delete the cluster using the command `minikube delete`.
